# Analise de sentimento

baseado no post do medium de: https://medium.com/@alegeorgelustosa/an%C3%A1lise-de-sentimentos-em-python-2a7d04a836e0

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk

##  Read csv file

In [2]:
df = pd.read_csv('./datasets/sentimentos.csv')

## View Data

In [3]:
df.sample(10)

,Texto,Sentimento
387,tenho aversão à gente chata,raiva
75,vou te chamar para comemorar,alegria
501,"nossa, que top esse filme!, eu nao esperava",surpresa
496,superou minhas expectativas,surpresa
123,meu pai esta adoentado,nojo
90,estamos tendo muito lucro,alegria
57,adoro este doce de frutas,alegria
622,"pra mim um dia e ruim, o outro e pior",tristeza
771,me sinto intimidada pela sua presença,medo
857,esta realmente deslumbrante querida,surpresa


In [4]:
df['Sentimento'].value_counts()

alegria     162
medo        156
raiva       152
tristeza    149
nojo        143
surpresa    141
Name: Sentimento, dtype: int64

In [5]:
df_alegria = df.loc[df.Sentimento == 'alegria'][0:140]
df_medo = df.loc[df.Sentimento == 'medo'][0:140]
df_raiva = df.loc[df.Sentimento == 'raiva'][0:140]
df_tristeza = df.loc[df.Sentimento == 'tristeza'][0:140]
df_nojo = df.loc[df.Sentimento == 'nojo'][0:140]
df_surpresa = df.loc[df.Sentimento == 'surpresa'][0:140]

In [6]:
df = pd.concat([df_alegria, df_medo, df_raiva, df_tristeza, df_nojo, df_surpresa], ignore_index=True, sort=True)

In [7]:
df.Sentimento.value_counts()

tristeza    140
medo        140
surpresa    140
raiva       140
alegria     140
nojo        140
Name: Sentimento, dtype: int64

## Load stop words 

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stop_words = nltk.corpus.stopwords.words('portuguese')

## Stemming: Removing sulfix and prefix of word

In [10]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [11]:
def apply_stemmer(text: str):
    stemmer = nltk.stem.RSLPStemmer()
    texts = []
    texts = [str(stemmer.stem(word)) for word in text.split() if word not in stop_words]
    return tuple(texts)

In [12]:
apply_stemmer('Ola admirável mundo novo, eu programei você!! Eu sou um programador')

('ola', 'admir', 'mund', 'novo,', 'program', 'você!!', 'eu', 'program')

In [13]:
df['Palavras'] = df['Texto'].apply(apply_stemmer)
df.sample(10)

,Sentimento,Texto,Palavras
639,nojo,que criança terrivelmente travessa,"(crianç, terri, travess)"
172,medo,tenho que correr pra não me pegarem,"(corr, pra, peg)"
42,alegria,estou cativada pelo seu olhar,"(cativ, olh)"
750,surpresa,opa! quem apagou a luz?,"(opa!, apag, luz?)"
557,tristeza,estou bastante aborrecido com o jornal,"(bast, aborrec, jorn)"
253,medo,que abominável esse montro!,"(abomin, montro!)"
455,tristeza,me aflige o modo como fala,"(aflig, mod, fal)"
657,nojo,o mundo e feio como o pecado,"(mund, fei, pec)"
52,alegria,seu amor e brilhante,"(am, brilh)"
808,surpresa,o cantor estava excelente,"(can, excel)"


## Getting words

In [14]:
words = []
for index, item in df.iterrows():
    words.extend(item.Palavras)

In [15]:
len(words)

2627

## Genereting the frequency distribution

In [16]:
frequency = nltk.FreqDist(words)
frequency.most_common(10)

[('vou', 30),
 ('nao', 30),
 ('tão', 21),
 ('quer', 21),
 ('tod', 20),
 ('tud', 18),
 ('ser', 18),
 ('fic', 17),
 ('sint', 17),
 ('pod', 14)]

## Getting unique words

In [17]:
unique_words = frequency.keys()
print(len(unique_words))

1245


In [18]:
def extract_words_to_text(text):
    doc = None
    if(type(text) == type(str)):
        doc = text.split()
    else:
        doc = set(text)
        
    features = {}
    for word in unique_words:
        features['%s' % word] = (word in doc)
    return features

In [19]:
def extract_data(df: pd.DataFrame):
    itens = []
    for i, row in df.iterrows():
        field = (tuple(row['Palavras']), row['Sentimento'])
        itens.append(field)
    return itens

In [20]:
extract_data(df)[0:5]

[(('trabalh', 'agrad'), 'alegria'),
 (('gost', 'fic', 'aconcheg'), 'alegria'),
 (('fiz', 'ades', 'curs', 'hoj', 'porqu', 'gost'), 'alegria'),
 (('admir', 'muit'), 'alegria'),
 (('ador',), 'alegria')]

## Splitting the data frame into train and test

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['Texto', 'Palavras']], df['Sentimento'], train_size=0.5, random_state=42)

In [22]:
y_train.value_counts()

surpresa    80
medo        75
raiva       71
nojo        71
alegria     63
tristeza    60
Name: Sentimento, dtype: int64

In [23]:
y_test.value_counts()

tristeza    80
alegria     77
raiva       69
nojo        69
medo        65
surpresa    60
Name: Sentimento, dtype: int64

In [24]:
X_train['Sentimento'] = y_train

## Train model

In [25]:
data = nltk.classify.apply_features(extract_words_to_text, extract_data(X_train), labeled=True)

In [26]:
classificador = nltk.NaiveBayesClassifier.train(data)

In [27]:
classificador.labels()

['medo', 'surpresa', 'raiva', 'tristeza', 'alegria', 'nojo']

In [28]:
classificador.show_most_informative_features(10)

Most Informative Features
                      am = True           alegri : surpre =      4.6 : 1.0
                     vou = True             medo : surpre =      4.6 : 1.0
                   perig = True             medo : nojo   =      4.1 : 1.0
                     tão = True           surpre : raiva  =      3.9 : 1.0
                     mim = True           triste : medo   =      3.7 : 1.0
                     fic = True             medo : surpre =      3.2 : 1.0
                     ser = True             medo : surpre =      3.2 : 1.0
                     tod = True            raiva : medo   =      3.2 : 1.0
                   terri = True             nojo : triste =      3.1 : 1.0
                 depress = True           triste : surpre =      3.1 : 1.0


## Evalueting model

In [29]:
print(nltk.classify.accuracy(classificador, data))

0.9380952380952381


In [30]:
def error_metric(data) -> []:
    erros = []
    for (words, target) in data:
        result = classificador.classify(words)
        if result != target:
            erros.append((target, result, words))
    print(f'Erros: {(round(len(erros) / len(data) * 100,2))} %')

In [31]:
error_metric(data)

Erros: 6.19 %


In [32]:
from nltk.metrics import ConfusionMatrix

def view_confusion_matrix(data):
    y = []
    pred = []
    for (word, target) in data:
        result = classificador.classify(word)
        y.append(target)
        pred.append(result)

    matriz = ConfusionMatrix(y, pred)
    print(matriz)

In [33]:
view_confusion_matrix(data)

         |              s  t |
         |  a           u  r |
         |  l           r  i |
         |  e        r  p  s |
         |  g  m  n  a  r  t |
         |  r  e  o  i  e  e |
         |  i  d  j  v  s  z |
         |  a  o  o  a  a  a |
---------+-------------------+
 alegria |<60> .  .  1  2  . |
    medo |  .<68> 2  2  3  . |
    nojo |  .  .<70> .  1  . |
   raiva |  .  2  .<69> .  . |
surpresa |  .  .  .  .<80> . |
tristeza |  .  2  1  3  7<47>|
---------+-------------------+
(row = reference; col = test)



## Teste model

In [34]:
X_test['Sentimento'] = y_test
data_test = nltk.classify.apply_features(extract_words_to_text, extract_data(X_test), labeled=True)

In [35]:
print(nltk.classify.accuracy(classificador, data_test))

0.37857142857142856


In [36]:
error_metric(data_test)

Erros: 62.14 %


In [37]:
view_confusion_matrix(data_test)

         |              s  t |
         |  a           u  r |
         |  l           r  i |
         |  e        r  p  s |
         |  g  m  n  a  r  t |
         |  r  e  o  i  e  e |
         |  i  d  j  v  s  z |
         |  a  o  o  a  a  a |
---------+-------------------+
 alegria |<20> 7  2 14 30  4 |
    medo |  3<27> 2  4 25  4 |
    nojo |  3  5<29> 8 20  4 |
   raiva |  4  6  7<22>25  5 |
surpresa |  3  6  2  5<42> 2 |
tristeza |  7 11  8  5 30<19>|
---------+-------------------+
(row = reference; col = test)



In [38]:
def predict_test(text):
    text = apply_stemmer(text)
    novo = extract_words_to_text(text)

    result = classificador.classify(novo)
    print('Predicação: %s' % result)

    print()
    distribuicao = classificador.prob_classify(novo)
    for clas in distribuicao.samples():
        print('%s: %f' % (clas, distribuicao.prob(clas)))

In [39]:
test = 'Eu vou ser pai!!!'
predict_test(test)

Predicação: medo

medo: 0.497480
surpresa: 0.102917
raiva: 0.183292
tristeza: 0.028040
alegria: 0.015175
nojo: 0.173095


In [40]:
test = 'Fui assaltado ontem'
predict_test(test)

Predicação: surpresa

medo: 0.167583
surpresa: 0.503812
raiva: 0.093970
tristeza: 0.044812
alegria: 0.074035
nojo: 0.115788


In [41]:
test = 'Ganhei na mega sena'
predict_test(test)

Predicação: surpresa

medo: 0.104242
surpresa: 0.334422
raiva: 0.168297
tristeza: 0.067909
alegria: 0.117758
nojo: 0.207371


In [42]:
test = 'Estou namorando uma gata linda'
predict_test(test)

Predicação: surpresa

medo: 0.015671
surpresa: 0.408074
raiva: 0.028210
tristeza: 0.048497
alegria: 0.393790
nojo: 0.105758
